In [8]:
SEED=42
import os
import glob
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.model_selection import KFold, GroupKFold
import lightgbm as lgb
import warnings
from utils import get_feature_groups
import itertools
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 300)
pd.set_option('max_rows', 300)

# Function to read our base train and test set
def read_train_test():
    train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
    test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
    # Create a key to merge with book and trade data
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    print(f'Our training set has {train.shape[0]} rows')
    return train, test

# Function to get group stats for the stock_id and time_id
def get_time_stock(df):
    # Get realized volatility columns
    vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 'log_return1_realized_volatility_450', 'log_return2_realized_volatility_450', 
                'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 'log_return1_realized_volatility_150', 'log_return2_realized_volatility_150', 
                'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_450', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_150']

    # Group by the stock id
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', 'median']).reset_index()
    # Rename columns joining suffix
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    # Group by the time id
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', 'median']).reset_index()
    # Rename columns joining suffix
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    # Merge with original dataframe
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df

# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

In [9]:
train, _ = read_train_test()
df_book = pd.read_csv('../input/processed-book-ffill/df_book.csv')
print(df_book.shape)
df_trade = pd.read_csv('../input/processed-book-ffill/df_trade.csv')
print(df_trade.shape)
train_ = df_book.merge(df_trade, on = ['row_id'], how = 'left')
train = train.merge(train_, on = ['row_id'], how = 'left')

# Get group stats of time_id and stock_id
train = get_time_stock(train)
train['logtarget'] = train['target'].apply(np.log)
train = train.sample(frac=1, random_state=SEED).reset_index(drop=True)
train

Our training set has 428932 rows
(428932, 249)
(428913, 57)


,stock_id,time_id,target,row_id,wap1_sum,wap1_mean,wap1_std,wap1_median,wap1_kurtosis,wap1_skew,wap2_sum,wap2_mean,wap2_std,wap2_median,wap2_kurtosis,wap2_skew,log_return1_sum,log_return1_realized_volatility,log_return1_mean,log_return1_std,log_return1_median,log_return1_kurtosis,log_return1_skew,log_return2_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_median,log_return2_kurtosis,log_return2_skew,wap_balance_sum,wap_balance_mean,wap_balance_std,wap_balance_median,wap_balance_kurtosis,wap_balance_skew,price_spread_sum,price_spread_mean,price_spread_std,price_spread_median,price_spread_kurtosis,price_spread_skew,bid_spread_sum,bid_spread_mean,bid_spread_std,bid_spread_median,bid_spread_kurtosis,bid_spread_skew,ask_spread_sum,ask_spread_mean,ask_spread_std,ask_spread_median,ask_spread_kurtosis,ask_spread_skew,total_volume_sum,total_volume_mean,total_volume_std,total_volume_median,total_volume_kurtosis,total_volume_skew,volume_imbalance_sum,volume_imbalance_mean,volume_imbalance_std,volume_imbalance_median,volume_imbalance_kurtosis,volume_imbalance_skew,wap1_sum_450,wap1_mean_450,wap1_std_450,wap1_median_450,wap1_kurtosis_450,wap1_skew_450,wap2_sum_450,wap2_mean_450,wap2_std_450,wap2_median_450,wap2_kurtosis_450,wap2_skew_450,log_return1_sum_450,log_return1_realized_volatility_450,log_return1_mean_450,log_return1_std_450,log_return1_median_450,log_return1_kurtosis_450,log_return1_skew_450,log_return2_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,log_return2_median_450,log_return2_kurtosis_450,log_return2_skew_450,wap_balance_sum_450,wap_balance_mean_450,wap_balance_std_450,wap_balance_median_450,wap_balance_kurtosis_450,wap_balance_skew_450,price_spread_sum_450,price_spread_mean_450,price_spread_std_450,price_spread_median_450,price_spread_kurtosis_450,price_spread_skew_450,bid_spread_sum_450,bid_spread_mean_450,bid_spread_std_450,bid_spread_median_450,bid_spread_kurtosis_450,bid_spread_skew_450,ask_spread_sum_450,ask_spread_mean_450,ask_spread_std_450,ask_spread_median_450,ask_spread_kurtosis_450,ask_spread_skew_450,total_volume_sum_450,total_volume_mean_450,total_volume_std_450,total_volume_median_450,total_volume_kurtosis_450,total_volume_skew_450,volume_imbalance_sum_450,volume_imbalance_mean_450,volume_imbalance_std_450,volume_imbalance_median_450,volume_imbalance_kurtosis_450,volume_imbalance_skew_450,wap1_sum_300,wap1_mean_300,wap1_std_300,wap1_median_300,wap1_kurtosis_300,wap1_skew_300,wap2_sum_300,wap2_mean_300,wap2_std_300,wap2_median_300,wap2_kurtosis_300,wap2_skew_300,log_return1_sum_300,log_return1_realized_volatility_300,log_return1_mean_300,log_return1_std_300,log_return1_median_300,log_return1_kurtosis_300,log_return1_skew_300,log_return2_sum_300,log_return2_realized_volatility_300,log_return2_mean_300,...,trade_order_count_skew_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_mean_300,trade_size_median_300,trade_size_std_300,trade_size_sum_300,trade_size_kurtosis_300,trade_size_skew_300,trade_order_count_mean_300,trade_order_count_median_300,trade_order_count_std_300,trade_order_count_sum_300,trade_order_count_kurtosis_300,trade_order_count_skew_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_mean_150,trade_size_median_150,trade_size_std_150,trade_size_sum_150,trade_size_kurtosis_150,trade_size_skew_150,trade_order_count_mean_150,trade_order_count_median_150,trade_order_count_std_150,trade_order_count_sum_150,trade_order_count_kurtosis_150,trade_order_count_skew_150,log_return1_realized_volatility_mean_stock,log_return1_realized_volatility_std_stock,log_return1_realized_volatility_max_stock,log_return1_realized_volatility_min_stock,log_return1_realized_volatility_median_stock,log_return2_realized_volatility_mean_stock,log_return2_realized_volatility_std_stock,log_return2_realized_volatility_max_stock,log_return2_realized_volatility_min_stock,log

In [10]:
train_ = pd.read_csv("../baseline/data/baseline_train.csv")
train = train.merge(train_, on=["time_id", "stock_id", "target"], how="inner")
train

,stock_id,time_id,target,row_id,wap1_sum,wap1_mean,wap1_std,wap1_median,wap1_kurtosis,wap1_skew,wap2_sum,wap2_mean,wap2_std,wap2_median,wap2_kurtosis,wap2_skew,log_return1_sum,log_return1_realized_volatility,log_return1_mean,log_return1_std,log_return1_median,log_return1_kurtosis,log_return1_skew,log_return2_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_median,log_return2_kurtosis,log_return2_skew,wap_balance_sum,wap_balance_mean,wap_balance_std,wap_balance_median,wap_balance_kurtosis,wap_balance_skew,price_spread_sum,price_spread_mean,price_spread_std,price_spread_median,price_spread_kurtosis,price_spread_skew,bid_spread_sum,bid_spread_mean,bid_spread_std,bid_spread_median,bid_spread_kurtosis,bid_spread_skew,ask_spread_sum,ask_spread_mean,ask_spread_std,ask_spread_median,ask_spread_kurtosis,ask_spread_skew,total_volume_sum,total_volume_mean,total_volume_std,total_volume_median,total_volume_kurtosis,total_volume_skew,volume_imbalance_sum,volume_imbalance_mean,volume_imbalance_std,volume_imbalance_median,volume_imbalance_kurtosis,volume_imbalance_skew,wap1_sum_450,wap1_mean_450,wap1_std_450,wap1_median_450,wap1_kurtosis_450,wap1_skew_450,wap2_sum_450,wap2_mean_450,wap2_std_450,wap2_median_450,wap2_kurtosis_450,wap2_skew_450,log_return1_sum_450,log_return1_realized_volatility_450,log_return1_mean_450,log_return1_std_450,log_return1_median_450,log_return1_kurtosis_450,log_return1_skew_450,log_return2_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,log_return2_median_450,log_return2_kurtosis_450,log_return2_skew_450,wap_balance_sum_450,wap_balance_mean_450,wap_balance_std_450,wap_balance_median_450,wap_balance_kurtosis_450,wap_balance_skew_450,price_spread_sum_450,price_spread_mean_450,price_spread_std_450,price_spread_median_450,price_spread_kurtosis_450,price_spread_skew_450,bid_spread_sum_450,bid_spread_mean_450,bid_spread_std_450,bid_spread_median_450,bid_spread_kurtosis_450,bid_spread_skew_450,ask_spread_sum_450,ask_spread_mean_450,ask_spread_std_450,ask_spread_median_450,ask_spread_kurtosis_450,ask_spread_skew_450,total_volume_sum_450,total_volume_mean_450,total_volume_std_450,total_volume_median_450,total_volume_kurtosis_450,total_volume_skew_450,volume_imbalance_sum_450,volume_imbalance_mean_450,volume_imbalance_std_450,volume_imbalance_median_450,volume_imbalance_kurtosis_450,volume_imbalance_skew_450,wap1_sum_300,wap1_mean_300,wap1_std_300,wap1_median_300,wap1_kurtosis_300,wap1_skew_300,wap2_sum_300,wap2_mean_300,wap2_std_300,wap2_median_300,wap2_kurtosis_300,wap2_skew_300,log_return1_sum_300,log_return1_realized_volatility_300,log_return1_mean_300,log_return1_std_300,log_return1_median_300,log_return1_kurtosis_300,log_return1_skew_300,log_return2_sum_300,log_return2_realized_volatility_300,log_return2_mean_300,...,waproll_std5_down50_skew,waproll_std5_down50_autocorr,waproll_std5_down50_median,waproll_std10_down50_mean,waproll_std10_down50_std,waproll_std10_down50_skew,waproll_std10_down50_autocorr,waproll_std10_down50_median,wapcandlef1_down50,wapcandlef2_down50,wapcandlef3_down50,wapcandlef41_down50,wapcandlef42_down50,up25num,waproll_std5_up25_mean,waproll_std5_up25_std,waproll_std5_up25_skew,waproll_std5_up25_autocorr,waproll_std5_up25_median,waproll_std10_up25_mean,waproll_std10_up25_std,waproll_std10_up25_skew,waproll_std10_up25_autocorr,waproll_std10_up25_median,wapcandlef1_up25,wapcandlef2_up25,wapcandlef3_up25,wapcandlef41_up25,wapcandlef42_up25,down25num,waproll_std5_down25_mean,waproll_std5_down25_std,waproll_std5_down25_skew,waproll_std5_down25_autocorr,waproll_std5_down25_median,waproll_std10_down25_mean,waproll_std10_down25_std,waproll_std10_down25_skew,waproll_std10_down25_autocorr,waproll_std10_down25_median,wapcandlef1_down25,wapcandlef2_down25,wapcandlef3_down25,wapcandlef41_down25,wapcandlef42_down25,up75num,waproll_std5_up75_mean,waproll_std5_up75_std,waproll_std5_up75_skew,waproll_std5_up75_autocorr,waproll_std5_up75_median,wapr

In [13]:
train['stock_id'] = train['stock_id'].astype('category')

In [14]:
# feature_groups = get_feature_groups(train)
# pruned_groups = ["timeagg_450_log_return1",
#     "trade_seconds_in_bucket_450",
#     "wap1_300",
#     "timeagg_450_trade_log_return",
#     "trade_seconds_in_bucket",
#     "ask_spread_450"]
# feature_cols = list(itertools.chain.from_iterable([c for g, c in feature_groups.items() if g not in pruned_groups]))
# print(f"# features: {len(feature_cols)}")
feature_cols = [c for c in train.columns if c not in ['row_id', 'target', 'time_id', 'target', 'logtarget']]
target_col = 'target'
# {'reg_alpha': 9.594283186777315e-06, 'reg_lambda': 0.00030328360027000927, 'n_estimators': 3178, 'learning_rate': 0.019233318286949023, 'num_leaves': 1251, 'feature_fraction': 0.26088472222018966, 'bagging_fraction': 0.20674682558678448, 'bagging_freq': 25, 'min_child_samples': 263}
params = {
    'random_state': SEED,
    'objective': 'rmse',  
    'boosting_type': 'gbdt',
#     'boosting_type': 'dart',
#     'num_leaves': 100,
#     'learning_rate': 0.1,
#     'colsample_bytree': 0.8,
#     'subsample': 0.8,
#     'n_estimators': 10000,
    'min_child_samples': int(0.01*train.shape[0])
}
# params = {
#     'n_estimators': 20000,
#     'objective': 'rmse', # poisson
#     'boosting_type': 'gbdt', # dart
#     'max_depth': -1,
#     'learning_rate': 0.01,
#     'subsample': 0.8,
#     'subsample_freq': 4,
#     'feature_fraction': 0.8,
#     'lambda_l1': 1,
#     'lambda_l2': 1,
#     'seed':SEED,
#     'early_stopping_rounds': 500,
#     'verbose': -1
# }

oof_predictions = np.zeros(train.shape[0])
kfold = GroupKFold(n_splits=5)
for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, groups=train.time_id)):
    print(f'Training fold {fold}')
    x_train, x_val = train.loc[trn_ind, feature_cols], train.loc[val_ind, feature_cols]
    y_train, y_val = train.loc[trn_ind, target_col], train.loc[val_ind, target_col]

    # Root mean squared percentage error weights
    if target_col == 'logtarget':
        train_weights = 1 / np.square(np.exp(y_train))
        val_weights = 1 / np.square(np.exp(y_val))
    else:
        train_weights = 1 / np.square(y_train)
        val_weights = 1 / np.square(y_val)

    # Fit with sklearn API
    model = lgb.LGBMRegressor(**params)
    model.fit(x_train, 
              y_train, 
              sample_weight=train_weights,
              eval_set=[(x_val, y_val)],
              eval_sample_weight=[val_weights],
              eval_metric='rmse',
              early_stopping_rounds=100,
              verbose=100)

    # Add predictions to the out of folds array
    oof_predictions[val_ind] = model.predict(x_val)
if target_col == 'logtarget':
    oof_predictions = np.exp(oof_predictions)
oof_score = rmspe(train['target'], oof_predictions)

print(f"oof rmspe: {oof_score}") # 0.2260 # dart 0.2465 #

Training fold 0
[LightGBM] [Warning] early_stopping_round is set=500, early_stopping_rounds=500 will be ignored. Current value: early_stopping_round=500
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
Training fold 1
[LightGBM] [Warning] early_stopping_round is set=500, early_stopping_rounds=500 will be ignored. Current value: early_stopping_round=500
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
Training fold 2
[LightGBM] [Warning] early_stopp

In [15]:
df_importance = pd.DataFrame({"feature": model.feature_name_, "importance": model.feature_importances_})
df_importance = df_importance.sort_values(by=["importance"], ascending=False).reset_index(drop=True)
df_importance

,feature,importance
0,stock_id,4467
1,__book_wap_diff_lambda_____median___time,598
2,trade_log_return_realized_volatility_450_media...,573
3,log_return2_realized_volatility_450_min_time,566
4,__book_wap_diff_lambda_____min___time,525
...,...,...
669,log_return2_realized_volatility_450_min_stock,0
670,log_return1_realized_volatility_300_max_stock,0
671,log_return2_realized_volatility_300_max_stock,0
672,waproll_std5_up25_autocorr,0
